In [1]:
import sys
sys.path.append('..')

import os

import numpy as np

from keras.optimizers import Nadam
from keras.callbacks import ModelCheckpoint, EarlyStopping

from src import models, preprocessing, utils

Using TensorFlow backend.


In [2]:
# Some useful parameters
KEY = 'word2vec_lstm'
DIR = os.path.join(utils.MODELS_DIR, KEY)

# Word2Vec parameters
size = 50      # dimensions of embedding space
window = 5     # number of words for Word2Vec
sg = 1         # whether to use skip-gram model (0: CBOW)
iter = 100     # number of epochs

# LSTM parameters
units = 1024
window_size = 11  # number of words for LSTM

### Load data

In [3]:
sonnets = utils.load_shakespeare()

### Pre-processing

Since we are training a character-based LSTM, we just need to map each character to a dimension.

In [4]:
encoding, encoded_sonnets = preprocessing.encode_words_word2vec(
    sonnets, size=size, window=window, sg=sg, iter=iter, workers=8
)
n_words = len(encoding.vocab)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Joseph\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
# Generate training data.
X = []
Y = []
for sonnet in encoded_sonnets:
    x = np.zeros((len(sonnet) - window_size, window_size, size))
    y = np.zeros((len(sonnet) - window_size, size))
    
    for i in range(len(sonnet) - window_size):
        x[i] = sonnet[i:i+window_size]
        y[i] = sonnet[i+window_size]
    X.append(x)
    Y.append(y)
X = np.vstack(X)
Y = np.vstack(Y)

### Define model

In [7]:
# Save model parameters.
params = {
    'units': units,
    'size': size,
    'sg': sg,
    'iter': iter,
    'window': window,
    'window_size': window_size,
    'encoding': encoding
}
utils.save_pickle(params, os.path.join(DIR, 'params.pkl'))

In [8]:
model = models.WordLSTM(units, window_size, size)
model.compile(loss='cosine_similarity', optimizer='adam')

In [9]:
# Save model with best accuracy.
save_path = os.path.join(DIR, 'model-{epoch:03d}-{loss:.4f}.h5')
checkpoint = ModelCheckpoint(save_path, monitor='loss', verbose=1, save_best_only=True, mode='min')
# Early stopping condition. Stop when loss has stopped decreasing for 10 epochs.
es = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=10)
model.fit(X, Y, epochs=1000, batch_size=64, callbacks=[checkpoint, es])

Epoch 1/1000
20846/20846 [==============================] - 7s 323us/step - loss: -0.1097

Epoch 00001: loss improved from inf to -0.10973, saving model to C:\Users\Joseph\Documents\GitHub\cs155-miniproject3\models\word2vec_lstm\model-001--0.1097.h5
Epoch 2/1000
20846/20846 [==============================] - 6s 297us/step - loss: -0.1164

Epoch 00002: loss improved from -0.10973 to -0.11638, saving model to C:\Users\Joseph\Documents\GitHub\cs155-miniproject3\models\word2vec_lstm\model-002--0.1164.h5
Epoch 3/1000
20846/20846 [==============================] - 6s 302us/step - loss: -0.1265

Epoch 00003: loss improved from -0.11638 to -0.12650, saving model to C:\Users\Joseph\Documents\GitHub\cs155-miniproject3\models\word2vec_lstm\model-003--0.1265.h5
Epoch 4/1000
20846/20846 [==============================] - 6s 298us/step - loss: -0.1287

Epoch 00004: loss improved from -0.12650 to -0.12872, saving model to C:\Users\Joseph\Documents\GitHub\cs155-miniproject3\models\word2vec_lstm\model-

KeyboardInterrupt: 